In [2]:
import os
import numpy as np
import tensorflow as tf
from data_loader import DataGenerator
from sklearn.metrics import f1_score, precision_score, recall_score
import datetime
from helpers import f1 as f1_metric
from model import lstm_unet

def set_random_seeds(seed):
    """Set random seeds for reproducibility."""
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

def create_data_generators():
    """Create data generators for training and validation."""
    train_gen = DataGenerator(
        image_dir="../Unet/data_v3_processed/train/images/images",
        mask_dir="../Unet/data_v3_processed/train/masks/masks",
        batch_size=8,
        time_steps=15,
        img_size=(256,256),
        normalize_images=True,
        normalize_masks=False,
        infinite=True,
    )
    
    val_gen = DataGenerator(
        image_dir="../Unet/data_v3_processed/val/images/images",
        mask_dir="../Unet/data_v3_processed/val/masks/masks",
        batch_size=8,
        time_steps=15,
        img_size=(256,256),
        normalize_images=True,
        normalize_masks=False,
        infinite=True
    )

    return train_gen, val_gen

def train_and_evaluate_model(run_number, train_generator, val_generator, 
                           epochs=3, steps_per_epoch=1000, validation_steps=5, 
                           random_seed=None):
    """Train a single model and return evaluation metrics."""
    print(f"\nTraining Run {run_number + 1}/5 (Seed: {random_seed})")
    print("=" * 50)
    
    if random_seed is not None:
        set_random_seeds(random_seed)
    
    model = lstm_unet()
    
    model.compile(optimizer='adam', 
                 loss='binary_crossentropy', 
                 metrics=[f1_metric])
    
    try:
        history = model.fit(train_generator,
                           validation_data=val_generator,
                           epochs=epochs,
                           steps_per_epoch=steps_per_epoch,
                           validation_steps=validation_steps,
                           verbose=1)

        model_path = os.path.join("models", f"lstm_unet_run{run_number+1}.h5")
        os.makedirs("models", exist_ok=True)
        model.save(model_path)
        
        return model_path
        
    except Exception as e:
        print(f"ERROR in training: {e}")
        return None

# Main execution
random_seeds = [789, 999]

for run in range(2):
    # CREATE FRESH GENERATORS FOR EACH RUN - this fixes the state issue
    train_generator, val_generator = create_data_generators()
    
    model_path = train_and_evaluate_model(
        run, train_generator, val_generator, random_seed=random_seeds[run]
    )
    
    tf.keras.backend.clear_session()


Training Run 1/5 (Seed: 789)
Epoch 1/3
1000/1000 [==============================] - 437s 419ms/step - loss: 0.0888 - f1: 0.5020 - val_loss: 0.0197 - val_f1: 0.5732
Epoch 2/3
1000/1000 [==============================] - 417s 416ms/step - loss: 0.0063 - f1: 0.9020 - val_loss: 0.0101 - val_f1: 0.7248
Epoch 3/3
1000/1000 [==============================] - 417s 416ms/step - loss: 0.0029 - f1: 0.9155 - val_loss: 0.0101 - val_f1: 0.6566

Training Run 2/5 (Seed: 999)
Epoch 1/3
1000/1000 [==============================] - 436s 418ms/step - loss: 0.0610 - f1: 0.5080 - val_loss: 0.0242 - val_f1: 0.4141
Epoch 2/3
1000/1000 [==============================] - 418s 417ms/step - loss: 0.0046 - f1: 0.9052 - val_loss: 0.0091 - val_f1: 0.7004
Epoch 3/3
1000/1000 [==============================] - 417s 416ms/step - loss: 0.0024 - f1: 0.9191 - val_loss: 0.0258 - val_f1: 0.3085
